In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import time
import csv
import pandas as pd

In [ ]:
data = pd.read_csv('../Streaming_Linux_process.csv')
data.head()

In [ ]:
def read_csv(fileName):
    data = []
    
    cRows = csv.DictReader(open(fileName))
    for rows in cRows:
        data.append(rows)
    return data


In [ ]:

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
if __name__ == '__main__':
    topic = 'process'
    process_rows = read_csv('../Streaming_Linux_process.csv')
    
    for i in range(len(process_rows)):  #Changing the datatypes of the elements to correct datatype
        process_rows[i]["sequence"] = int(process_rows[i]["sequence"])
        process_rows[i]["machine"] = int(process_rows[i]["machine"])
        process_rows[i]["PID"] = int(process_rows[i]["PID"])
        process_rows[i]["TRUN"] = int(process_rows[i]["TRUN"])
        process_rows[i]["TSLPI"] = int(process_rows[i]["TSLPI"])
        process_rows[i]["TSLPU"] = int(process_rows[i]["TSLPU"])
        process_rows[i]["POLI"] = str(process_rows[i]["POLI"])
        process_rows[i]["NICE"] = int(process_rows[i]["NICE"])
        process_rows[i]["PRI"] = int(process_rows[i]["PRI"])
        process_rows[i]["RTPR"] = int(process_rows[i]["RTPR"])
        process_rows[i]["CPUNR"] = int(process_rows[i]["CPUNR"])
        process_rows[i]["EXC"] = int(process_rows[i]["EXC"])
        process_rows[i]["Status"] = str(process_rows[i]["Status"])
        process_rows[i]["State"] = str(process_rows[i]["State"])
        process_rows[i]["CPU"] = float(process_rows[i]["CPU"])
        process_rows[i]["CMD"] = str(process_rows[i]["CMD"])

    
    print('Publishing records..')
    producer = connect_kafka_producer()
#     print(process_rows[0])
    
    m4,m5,m6,m7,m8 = [],[],[],[],[]
    for record in process_rows: 
#         print(type(record['machine']))
        if(record['machine'] == 4):
            m4.append(record)
        elif(record['machine'] == 5):
            m5.append(record)
        elif(record['machine'] == 6):
            m6.append(record)
        elif(record['machine'] == 7):
            m7.append(record)
        elif(record['machine'] == 8):
            m8.append(record)
            
    print('m4', m4)
    x4 = x5 = x6 = x7 = x8 = 0
#     print(m5[0])
    while(True):
        try:
            data = []
            ts = int(dt.datetime.utcnow().timestamp()) 
            randint = random.randint(10,50)
            data+=m4[x4:x4+randint]

            randomint = random.randint(10,50)
            try:
                tempdata = m5[x5:x5 +randomint]
                for x in tempdata:
                    x['ts'] = ts
                data+=(tempdata)
                x5+=randomint
            except:
                None

            randomint = random.randint(10,50)
            try:
                tempdata = m6[x6:x6 +randomint]
                for x in tempdata:
                    x['ts'] = ts
                data+=(tempdata)
                x6+=randomint
            except: 
                None

            randomint = random.randint(10,50)
            try:
                tempdata = m7[x7:x7 +randomint]
                for x in tempdata:
                    x['ts'] = ts
                data+=(tempdata)
                x7+=randomint
            except: 
                None

            randomint = random.randint(10,50)
            try:
                tempdata = m8[x8:x8 +randomint]
                for x in tempdata:
                    x['ts'] = ts
                data+=(tempdata)
                x8+=randomint
            except: 
                None


            publish_message(producer, topic, data)
            
            time.sleep(5)
            
            if(len(m4<=x4) and len(m5<=x5) and len(m6<=x6) and len(m7<=x7) and len(m8<=x4))
            x4 = x5 = x6 = x7 = x8 = 0
        except Exception as e:
            print(e)
#             x4 = x5 = x6 = x7 = x8 = 0

    